# Challenge Técnico – Sr Data Engineer

¡Felicitaciones!

Si estás leyendo esto, es porque llegaste a una etapa muy importante de nuestro proceso de selección.  
Te invitamos a desarrollar nuestro **Challenge Técnico** para la posición de **Sr Data Engineer**.

---

## ¿Por qué esta etapa es importante?

Porque nos ayuda a realizar la próxima etapa (entrevista técnica) con mayor objetividad, pero principalmente nos aporta información muy valiosa sobre tus **hard skills**.

---

## ¿Cuánto tiempo tengo para realizar el Challenge?

Tienes **5 días corridos** para realizarlo.  
Está pensado para invertir aproximadamente **1 hora por día**, considerando que también tienes otras responsabilidades laborales y personales.

---

## ¿Qué sucede si no realizo el Challenge?

Lamentablemente, no podremos continuar con el proceso, ya que se trata de una de las instancias **más importantes y definitorias**.

---

¡Mucho éxito!

---

## 📌 Challenge

### 1. Dataset
- Descargar el dataset **Covid-19** desde Kaggle:  
  https://www.kaggle.com/datasets/imdevskp/corona-virus-report

---

### 2. Procesamiento con Spark
- Cargar los datasets utilizando **Apache Spark**.
- Mantener los datos en formato **Parquet**.
- Utilizar **paralelismo y RDD**.

---

### 3. Procesamiento con Pandas
- Cargar los datasets utilizando **Pandas**.
- Mantener los datos en formato **Parquet**.

---

### 4. Modelado
- Desarrollar un **diagrama DER (Diagrama Entidad-Relación)** del modelado de datos.

---

## 📦 Entregables

- Entregar el **proceso y los datos en un archivo ZIP**.
  - No es necesario entregar el ambiente armado.
- El desafío **puede publicarse en GitHub** (opcional).

---

## ⚙️ Consideraciones Técnicas

### Spark
- Para realizar el desafío con Spark es necesario instalar un **Spark Cluster**.
- Recursos útiles:
  - https://towardsdatascience.com/spark-on-windows-a-getting-started-guide-11dc44412164
  - https://www.geeksforgeeks.org/install-apache-spark-in-a-standalone-mode-on-windows/
  - https://medium.com/codex/setup-a-spark-cluster-step-by-step-in-10-minutes-922c06f8e2b1
- Para Spark, **se debe utilizar RDD**.

---

### Código y Diseño
- El código debe:
  - Tener **comentarios en las partes importantes**.
  - Estar desarrollado de forma **orientada a objetos** (uso de **clases**, incluso dentro de notebooks).

---

### Calidad de Datos
- Tratar correctamente los **tipos de datos en la última capa**.
  - Ejemplo: todo lo que sea numérico debe ser **numérico**, no string.

---

### Control de Datos
- Implementar un **control de datos**:
  - Para cada procesamiento, obtener **solo las novedades**.
  - Utilizar un esquema de **offset / delta de datos** (no reprocesar todo).

---

## 🧑‍💻 Lenguaje
- El desafío puede realizarse en **Python o Scala**.

---


In [9]:
import os
import shutil

base = "/content/covid_challenge"

for folder in ["parquet_spark", "parquet_pandas", "state"]:
    path = os.path.join(base, folder)
    if os.path.exists(path):
        shutil.rmtree(path)


In [10]:
# ============================================================
# Colab: Sr Data Engineer Challenge - End-to-end Implementation
# Dataset: https://www.kaggle.com/datasets/imdevskp/corona-virus-report
#
# Key goals (per PDF):
# - Download dataset from Kaggle
# - Load datasets with Spark (use parallelism + RDD) and store as Parquet
# - Load datasets with Pandas and store as Parquet
# - OOP approach (classes in notebook)
# - Type handling in the last layer
# - Delta/offset control: process only "novelties" (new rows per run)
# - Comment important parts in English and keep code clean / SOLID
# ============================================================

from __future__ import annotations

import os
import re
import json
import shutil
import hashlib
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple, Iterable
import zipfile
from pathlib import Path

# ----------------------------
# 1) Install dependencies
# ----------------------------
!pip -q install kaggle pyarrow fastparquet

# Spark in Colab
!apt-get -qq update
!apt-get -qq install -y openjdk-11-jdk-headless > /dev/null
!pip -q install pyspark

# ----------------------------
# 2) Kaggle auth setup
# ----------------------------
from google.colab import files

def _ensure_kaggle_credentials(kaggle_dir: str = "/root/.kaggle") -> None:
    """
    Input:
      - kaggle_dir: folder where kaggle.json must exist
    Output:
      - None. Ensures Kaggle credentials are present with correct permissions.
    """
    os.makedirs(kaggle_dir, exist_ok=True)
    kaggle_json_path = os.path.join(kaggle_dir, "kaggle.json")

    if os.path.exists(kaggle_json_path):
        os.chmod(kaggle_json_path, 0o600)
        print("Kaggle credentials already present.")
        return

    print("Upload your Kaggle API token file (kaggle.json) to continue.")
    uploaded = files.upload()

    if not uploaded:
        raise RuntimeError("No file uploaded. Please click 'Choose files' and select kaggle.json.")

    # If the user uploads kaggle.json with a different name (common in Colab),
    # we pick the first .json file and rename it to kaggle.json.
    json_files = [name for name in uploaded.keys() if name.lower().endswith(".json")]
    if not json_files:
        raise RuntimeError(f"Uploaded file(s) are not JSON: {list(uploaded.keys())}. Please upload kaggle.json.")

    uploaded_name = json_files[0]

    # Move/rename to the expected location
    shutil.move(uploaded_name, kaggle_json_path)
    os.chmod(kaggle_json_path, 0o600)

    print(f"Kaggle credentials ready: {kaggle_json_path}")

_ensure_kaggle_credentials()

# ----------------------------
# 3) Spark imports
# ----------------------------
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql import types as T


# ============================================================
# Clean Code / SOLID: Core domain model
# ============================================================

@dataclass(frozen=True)
class AppConfig:
    """
    Input:
      - base_dir: root folder for data outputs
    Output:
      - Provides normalized paths for raw/processed/state layers.
    """
    base_dir: str = "/content/covid_challenge"

    @property
    def raw_dir(self) -> str:
        return os.path.join(self.base_dir, "raw")

    @property
    def spark_parquet_dir(self) -> str:
        return os.path.join(self.base_dir, "parquet_spark")

    @property
    def pandas_parquet_dir(self) -> str:
        return os.path.join(self.base_dir, "parquet_pandas")

    @property
    def state_dir(self) -> str:
        return os.path.join(self.base_dir, "state")

    def ensure_dirs(self) -> None:
        os.makedirs(self.raw_dir, exist_ok=True)
        os.makedirs(self.spark_parquet_dir, exist_ok=True)
        os.makedirs(self.pandas_parquet_dir, exist_ok=True)
        os.makedirs(self.state_dir, exist_ok=True)


class SparkSessionFactory:
    """
    Single Responsibility:
      - Create and configure SparkSession.
    """

    def create(self, app_name: str = "SrDataEngineerCovidChallenge") -> SparkSession:
        """
        Input:
          - app_name: Spark application name
        Output:
          - SparkSession ready to use
        """
        spark = (
            SparkSession.builder
            .appName(app_name)
            .master("local[*]")
            # Practical settings for local/colab execution
            .config("spark.sql.shuffle.partitions", "8")
            .config("spark.default.parallelism", "8")
            .config("spark.sql.execution.arrow.pyspark.enabled", "true")
            .getOrCreate()
        )
        spark.sparkContext.setLogLevel("WARN")
        return spark


class KaggleDownloader:
    """
    Single Responsibility:
      - Download and unzip Kaggle dataset into raw_dir.
    """

    def __init__(self, dataset: str) -> None:
        self._dataset = dataset

    def download_and_extract(self, raw_dir: str) -> str:
        """
        Input:
          - raw_dir: destination folder for raw files
        Output:
          - path to the folder containing extracted CSVs
        """
        # Optional: keep raw_dir clean to avoid picking an old zip
        for f in os.listdir(raw_dir):
          if f.endswith(".zip"):
            os.remove(os.path.join(raw_dir, f))

        os.makedirs(raw_dir, exist_ok=True)

        # Download full dataset zip (no -f, because -f is for a specific file inside the dataset)
        !kaggle datasets download -d {self._dataset} -p {raw_dir} -q

        # Kaggle names the zip like "<dataset-slug>.zip"
        zips = [os.path.join(raw_dir, f) for f in os.listdir(raw_dir) if f.endswith(".zip")]
        if not zips:
            raise RuntimeError("Dataset zip not found after Kaggle download.")

        zip_path = zips[0]

        extract_dir = os.path.join(raw_dir, "extracted")
        os.makedirs(extract_dir, exist_ok=True)

        import zipfile
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_dir)

        print(f"Extracted dataset to: {extract_dir}")
        return extract_dir


@dataclass(frozen=True)
class DatasetSpec:
    """
    Defines how to process a dataset.
    """
    name: str                 # logical name (used for output folders)
    csv_path: str             # absolute path to the CSV
    key_columns: List[str]    # columns used to define uniqueness for delta control
    schema: Dict[str, str]    # last-layer types: spark SQL types as strings (e.g., 'int', 'double', 'date')


class DatasetCatalog:
    """
    Single Responsibility:
      - Discover CSVs in extracted folder and produce DatasetSpec objects.
    Notes:
      - Key columns are chosen to be stable and practical for delta tracking.
      - Type casting is applied at the 'last layer' before writing Parquet.
    """

    def build(self, extracted_dir: str) -> List[DatasetSpec]:
        """
        Input:
          - extracted_dir: folder with CSV files
        Output:
          - list of DatasetSpec for each known dataset
        """
        files = {f.lower(): os.path.join(extracted_dir, f) for f in os.listdir(extracted_dir) if f.lower().endswith(".csv")}

        def pick(*candidates: str) -> Optional[str]:
            for c in candidates:
                if c.lower() in files:
                    return files[c.lower()]
            return None

        # Known files in this Kaggle dataset (can change slightly; handle gracefully)
        specs: List[DatasetSpec] = []

        # Helper schemas (keep explicit to show type handling)
        date_schema = {
            "Date": "date",
        }

        # covid_19_clean_complete.csv
        p = pick("covid_19_clean_complete.csv")
        if p:
            specs.append(DatasetSpec(
                name="covid_19_clean_complete",
                csv_path=p,
                key_columns=["Date", "Country/Region", "Province/State"],
                schema={
                    "Date": "date",
                    "Province/State": "string",
                    "Country/Region": "string",
                    "Lat": "double",
                    "Long": "double",
                    "Confirmed": "long",
                    "Deaths": "long",
                    "Recovered": "long",
                    "Active": "long",
                    "WHO Region": "string",
                }
            ))

        # country_wise_latest.csv
        p = pick("country_wise_latest.csv")
        if p:
            specs.append(DatasetSpec(
                name="country_wise_latest",
                csv_path=p,
                key_columns=["Country/Region"],
                schema={
                    "Country/Region": "string",
                    "Confirmed": "long",
                    "Deaths": "long",
                    "Recovered": "long",
                    "Active": "long",
                    "New cases": "long",
                    "New deaths": "long",
                    "New recovered": "long",
                    "Deaths / 100 Cases": "double",
                    "Recovered / 100 Cases": "double",
                    "Deaths / 100 Recovered": "double",
                    "Confirmed last week": "long",
                    "1 week change": "long",
                    "1 week % increase": "double",
                    "WHO Region": "string",
                }
            ))

        # day_wise.csv
        p = pick("day_wise.csv")
        if p:
            specs.append(DatasetSpec(
                name="day_wise",
                csv_path=p,
                key_columns=["Date"],
                schema={
                    "Date": "date",
                    "Confirmed": "long",
                    "Deaths": "long",
                    "Recovered": "long",
                    "Active": "long",
                    "New cases": "long",
                    "New deaths": "long",
                    "New recovered": "long",
                    "Deaths / 100 Cases": "double",
                    "Recovered / 100 Cases": "double",
                    "Deaths / 100 Recovered": "double",
                    "No. of countries": "long",
                }
            ))

        # full_grouped.csv
        p = pick("full_grouped.csv")
        if p:
            specs.append(DatasetSpec(
                name="full_grouped",
                csv_path=p,
                key_columns=["Date", "Country/Region"],
                schema={
                    "Date": "date",
                    "Country/Region": "string",
                    "Confirmed": "long",
                    "Deaths": "long",
                    "Recovered": "long",
                    "Active": "long",
                    "New cases": "long",
                    "New deaths": "long",
                    "New recovered": "long",
                    "WHO Region": "string",
                }
            ))

        # usa_county_wise.csv
        p = pick("usa_county_wise.csv")
        if p:
            specs.append(DatasetSpec(
                name="usa_county_wise",
                csv_path=p,
                key_columns=["UID"],  # UID is the most stable unique identifier in this file
                schema={
                    "UID": "long",
                    "iso2": "string",
                    "iso3": "string",
                    "code3": "string",
                    "FIPS": "string",
                    "Admin2": "string",
                    "Province_State": "string",
                    "Country_Region": "string",
                    "Lat": "double",
                    "Long_": "double",
                    "Combined_Key": "string",
                    "Confirmed": "long",
                    "Deaths": "long",
                }
            ))

        # worldometer_data.csv
        p = pick("worldometer_data.csv")
        if p:
            specs.append(DatasetSpec(
                name="worldometer_data",
                csv_path=p,
                key_columns=["Country/Region"],
                schema={
                    "Country/Region": "string",
                    "Continent": "string",
                    "Population": "long",
                    "TotalCases": "long",
                    "NewCases": "long",
                    "TotalDeaths": "long",
                    "NewDeaths": "long",
                    "TotalRecovered": "long",
                    "NewRecovered": "long",
                    "ActiveCases": "long",
                    "Serious,Critical": "long",
                    "Tot Cases/1M pop": "double",
                    "Deaths/1M pop": "double",
                    "TotalTests": "long",
                    "Tests/1M pop": "double",
                    "WHO Region": "string",
                }
            ))

        if not specs:
            raise RuntimeError("No known CSVs found in extracted dataset folder.")

        return specs


class SparkSchemaCaster:
    """
    Single Responsibility:
      - Apply strict typing at the last layer (before writing Parquet).
    """

    _TYPE_MAP: Dict[str, T.DataType] = {
        "string": T.StringType(),
        "int": T.IntegerType(),
        "long": T.LongType(),
        "double": T.DoubleType(),
        "date": T.DateType(),
        "timestamp": T.TimestampType(),
        "boolean": T.BooleanType(),
    }

    def cast(self, df: DataFrame, schema: Dict[str, str]) -> DataFrame:
        """
        Input:
          - df: Spark DataFrame with raw columns
          - schema: dict of column -> type (string names)
        Output:
          - DataFrame with casted columns (when present)
        """
        out = df

        # Important: CSV inputs often contain columns with spaces, slashes and dots.
        # We must always escape column names using backticks to avoid Spark parsing issues.
        for col_name, type_name in schema.items():
            if col_name not in out.columns:
                continue

            if type_name not in self._TYPE_MAP:
                raise ValueError(f"Unsupported type '{type_name}' for column '{col_name}'")

            spark_type = self._TYPE_MAP[type_name]

            # Always use escaped column reference
            c = F.col(f"`{col_name}`")

            if type_name == "date":
                out = out.withColumn(col_name, F.to_date(c))
            else:
                out = out.withColumn(
                    col_name,
                    F.regexp_replace(c.cast("string"), ",", "").cast(spark_type)
                )

        return out

class DeltaStateStore:
    """
    Single Responsibility:
      - Persist and load the "already processed keys" for each dataset.
    Design choice:
      - We store a compact SHA-256 hash of the composite key to minimize storage.
    """

    def __init__(self, spark: SparkSession, state_dir: str) -> None:
        self._spark = spark
        self._state_dir = state_dir
        os.makedirs(self._state_dir, exist_ok=True)

    def _state_path(self, dataset_name: str) -> str:
        return os.path.join(self._state_dir, f"{dataset_name}_seen_keys.parquet")

    def load_seen_keys(self, dataset_name: str) -> Optional[DataFrame]:
        """
        Input:
          - dataset_name
        Output:
          - DataFrame with column 'row_key_hash' if exists; otherwise None
        """
        path = self._state_path(dataset_name)
        if not os.path.exists(path):
            return None
        return self._spark.read.parquet(path)

    def save_seen_keys(self, dataset_name: str, keys_df: DataFrame) -> None:
        """
        Input:
          - dataset_name
          - keys_df: DataFrame with column 'row_key_hash'
        Output:
          - None. Writes Parquet overwriting previous state.
        """
        path = self._state_path(dataset_name)
        keys_df.select("row_key_hash").dropDuplicates().write.mode("overwrite").parquet(path)


class SparkCovidIngestor:
    """
    Single Responsibility:
      - Read CSV with Spark, use RDD for parallelism, apply delta logic, cast types, write Parquet.
    """

    def __init__(
        self,
        spark: SparkSession,
        caster: SparkSchemaCaster,
        state_store: DeltaStateStore,
        output_dir: str,
    ) -> None:
        self._spark = spark
        self._caster = caster
        self._state_store = state_store
        self._output_dir = output_dir
        os.makedirs(self._output_dir, exist_ok=True)

    def _compute_row_key_hash(self, df: DataFrame, key_columns: List[str]) -> DataFrame:
        """
        Input:
          - df: DataFrame with raw columns
          - key_columns: list of columns that define uniqueness
        Output:
          - DataFrame with an extra 'row_key_hash' column
        """
        # Concatenate keys safely (null-safe) and hash.
        # Using SHA-256 to reduce collision risk.
        concat_expr = F.concat_ws(
            "||",
            *[F.coalesce(F.col(c).cast("string"), F.lit("")) for c in key_columns]
        )
        return df.withColumn("row_key_hash", F.sha2(concat_expr, 256))

    def _read_csv_with_rdd(self, csv_path: str) -> DataFrame:
        """
        Input:
          - csv_path: path to CSV file
        Output:
          - DataFrame loaded from CSV using an RDD-based approach for the file content
        Why RDD here:
          - The challenge explicitly requests using RDD.
          - We read the file as text via SparkContext.textFile (RDD), then parse with Spark CSV reader.
        """
        sc = self._spark.sparkContext

        # RDD parallel read of file content
        rdd = sc.textFile(csv_path)

        # Create a DataFrame from the RDD lines via spark.read.csv on an RDD
        # Note: Spark supports reading from an RDD[String] with header inference.
        df = (
            self._spark.read
            .option("header", True)
            .option("inferSchema", False)  # we enforce types later
            .option("multiLine", False)
            .csv(rdd)
        )
        return df

    def ingest(self, spec: DatasetSpec) -> Tuple[int, int]:
        """
        Input:
          - spec: DatasetSpec defining file, keys, schema
        Output:
          - (total_rows_read, delta_rows_written)
        """
        df_raw = self._read_csv_with_rdd(spec.csv_path)
        total_rows = df_raw.count()

        # Delta logic: keep only rows whose composite key hash was never seen before
        df_with_keys = self._compute_row_key_hash(df_raw, spec.key_columns)
        seen = self._state_store.load_seen_keys(spec.name)

        if seen is not None:
            df_delta = df_with_keys.join(seen, on="row_key_hash", how="left_anti")
        else:
            df_delta = df_with_keys

        # Cast at last layer (before writing)
        df_typed = self._caster.cast(df_delta, spec.schema)

        # Write delta parquet (append)
        out_path = os.path.join(self._output_dir, spec.name)
        df_typed.write.mode("append").parquet(out_path)

        delta_rows = df_typed.count()

        # Update state with newly processed keys
        new_keys = df_with_keys.select("row_key_hash").dropDuplicates()
        if seen is not None:
            merged = seen.unionByName(new_keys).dropDuplicates(["row_key_hash"])
        else:
            merged = new_keys
        self._state_store.save_seen_keys(spec.name, merged)

        return total_rows, delta_rows


# ============================================================
# Pandas pipeline (separate responsibility, same outputs)
# ============================================================

import pandas as pd

class PandasSchemaCaster:
    """
    Single Responsibility:
      - Apply final typing in Pandas pipeline before writing Parquet.
    """

    def cast(self, pdf: pd.DataFrame, schema: Dict[str, str]) -> pd.DataFrame:
        """
        Input:
          - pdf: Pandas DataFrame
          - schema: dict of column -> type string
        Output:
          - DataFrame with casted columns (when possible)
        """
        out = pdf.copy()

        for col, type_name in schema.items():
            if col not in out.columns:
                continue

            if type_name in ("int", "long"):
                out[col] = pd.to_numeric(out[col].astype(str).str.replace(",", ""), errors="coerce").astype("Int64")
            elif type_name == "double":
                out[col] = pd.to_numeric(out[col].astype(str).str.replace(",", ""), errors="coerce")
            elif type_name == "date":
                out[col] = pd.to_datetime(out[col], errors="coerce").dt.date
            elif type_name == "boolean":
                out[col] = out[col].astype("boolean")
            else:
                out[col] = out[col].astype("string")

        return out


class PandasDeltaStateStore:
    """
    Single Responsibility:
      - Persist seen keys for Pandas pipeline (same idea as Spark state).
    """

    def __init__(self, state_dir: str) -> None:
        self._state_dir = state_dir
        os.makedirs(self._state_dir, exist_ok=True)

    def _state_path(self, dataset_name: str) -> str:
        return os.path.join(self._state_dir, f"{dataset_name}_seen_keys_pandas.parquet")

    def load_seen(self, dataset_name: str) -> Optional[pd.DataFrame]:
        path = self._state_path(dataset_name)
        if not os.path.exists(path):
            return None
        return pd.read_parquet(path)

    def save_seen(self, dataset_name: str, seen_df: pd.DataFrame) -> None:
        path = self._state_path(dataset_name)
        seen_df[["row_key_hash"]].drop_duplicates().to_parquet(path, index=False)


class PandasCovidIngestor:
    """
    Single Responsibility:
      - Read CSV with Pandas, apply delta logic, cast types, write Parquet.
    """

    def __init__(
        self,
        caster: PandasSchemaCaster,
        state_store: PandasDeltaStateStore,
        output_dir: str,
    ) -> None:
        self._caster = caster
        self._state_store = state_store
        self._output_dir = output_dir
        os.makedirs(self._output_dir, exist_ok=True)

    def _compute_row_key_hash(self, pdf: pd.DataFrame, key_columns: List[str]) -> pd.DataFrame:
        """
        Input:
          - pdf: raw Pandas DataFrame
          - key_columns: columns that define uniqueness
        Output:
          - DataFrame with an extra 'row_key_hash' column
        """
        out = pdf.copy()

        # Null-safe string concatenation for composite keys
        # We keep it simple and deterministic so it matches run-to-run.
        def _row_key(row: pd.Series) -> str:
            parts = []
            for c in key_columns:
                v = row.get(c, "")
                if pd.isna(v):
                    v = ""
                parts.append(str(v))
            return "||".join(parts)

        keys = out.apply(_row_key, axis=1)
        out["row_key_hash"] = keys.apply(lambda s: hashlib.sha256(s.encode("utf-8")).hexdigest())
        return out

    def ingest(self, spec: DatasetSpec) -> Tuple[int, int]:
        """
        Input:
          - spec: DatasetSpec defining file, keys, schema
        Output:
          - (total_rows_read, delta_rows_written)
        """
        pdf_raw = pd.read_csv(spec.csv_path)
        total_rows = int(len(pdf_raw))

        pdf_with_keys = self._compute_row_key_hash(pdf_raw, spec.key_columns)
        seen = self._state_store.load_seen(spec.name)

        if seen is not None and "row_key_hash" in seen.columns:
            # Keep only novelties (rows never processed before)
            pdf_delta = pdf_with_keys[~pdf_with_keys["row_key_hash"].isin(set(seen["row_key_hash"].tolist()))].copy()
        else:
            pdf_delta = pdf_with_keys

        # Type enforcement at the last layer (before writing parquet)
        pdf_typed = self._caster.cast(pdf_delta, spec.schema)

        out_path = os.path.join(self._output_dir, spec.name)
        os.makedirs(out_path, exist_ok=True)

        # Write one parquet file per run to keep append semantics simple in Colab
        run_id = hashlib.md5(str(pd.Timestamp.utcnow()).encode("utf-8")).hexdigest()[:10]
        parquet_path = os.path.join(out_path, f"part_{run_id}.parquet")
        pdf_typed.to_parquet(parquet_path, index=False)

        delta_rows = int(len(pdf_typed))

        # Update state (append new keys into the "seen" set)
        new_keys = pdf_with_keys[["row_key_hash"]].drop_duplicates()

        if seen is not None and "row_key_hash" in seen.columns:
            merged = pd.concat([seen[["row_key_hash"]], new_keys], ignore_index=True).drop_duplicates()
        else:
            merged = new_keys

        self._state_store.save_seen(spec.name, merged)

        return total_rows, delta_rows


# ============================================================
# Orchestration layer (keeps notebook clean and testable)
# ============================================================

class PipelineRunner:
    """
    Single Responsibility:
      - Orchestrate the end-to-end pipeline steps using injected dependencies.
    Design:
      - Keeps the "main" flow readable and avoids mixing I/O with business logic.
    """

    def __init__(
        self,
        config: AppConfig,
        downloader: KaggleDownloader,
        catalog: DatasetCatalog,
        spark_factory: SparkSessionFactory,
    ) -> None:
        self._config = config
        self._downloader = downloader
        self._catalog = catalog
        self._spark_factory = spark_factory

    def run(self) -> None:
        """
        Input:
          - None (uses config and injected collaborators)
        Output:
          - None. Writes parquet outputs and state to disk.
        """
        self._config.ensure_dirs()

        extracted_dir = self._downloader.download_and_extract(self._config.raw_dir)
        specs = self._catalog.build(extracted_dir)

        spark = self._spark_factory.create()

        try:
            # Spark pipeline components
            spark_caster = SparkSchemaCaster()
            spark_state = DeltaStateStore(spark=spark, state_dir=self._config.state_dir)
            spark_ingestor = SparkCovidIngestor(
                spark=spark,
                caster=spark_caster,
                state_store=spark_state,
                output_dir=self._config.spark_parquet_dir,
            )

            # Pandas pipeline components
            pandas_caster = PandasSchemaCaster()
            pandas_state = PandasDeltaStateStore(state_dir=self._config.state_dir)
            pandas_ingestor = PandasCovidIngestor(
                caster=pandas_caster,
                state_store=pandas_state,
                output_dir=self._config.pandas_parquet_dir,
            )

            print("============================================================")
            print("Running ingestion (Spark + RDD) and ingestion (Pandas)")
            print("Delta logic: only novelties are appended each run")
            print("============================================================")

            spark_report = []
            pandas_report = []

            for spec in specs:
                # Spark ingestion
                total_s, delta_s = spark_ingestor.ingest(spec)
                spark_report.append((spec.name, total_s, delta_s))

                # Pandas ingestion
                total_p, delta_p = pandas_ingestor.ingest(spec)
                pandas_report.append((spec.name, total_p, delta_p))

            print("\n--- Spark report (RDD-based CSV read) ---")
            for name, total, delta in spark_report:
                print(f"[Spark] {name}: read={total} | delta_written={delta}")

            print("\n--- Pandas report ---")
            for name, total, delta in pandas_report:
                print(f"[Pandas] {name}: read={total} | delta_written={delta}")

            # Optional: quick sanity check showing one dataset schema after typing
            self._print_sample_schema(spark, specs)

            # Create deliverable zip
            zip_path = self._build_zip_deliverable()
            print(f"\nDeliverable ZIP created at: {zip_path}")

            # Provide DER diagram (as text artifact)
            self._print_der_diagram()

        finally:
            # Always close Spark cleanly
            print("\nStopping Spark session...")
            spark.stop()
            print("Spark stopped.")

    def _print_sample_schema(self, spark: SparkSession, specs: List[DatasetSpec]) -> None:
        """
        Input:
          - spark: active SparkSession
          - specs: dataset specs list
        Output:
          - None. Prints one schema as a quick verification.
        """
        # Pick the first dataset that exists in Spark output
        if not specs:
            return

        candidate = specs[0].name
        path = os.path.join(self._config.spark_parquet_dir, candidate)

        if os.path.exists(path):
            print(f"\nSchema check (Spark parquet): {candidate}")
            df = spark.read.parquet(path)
            df.printSchema()
            print("\nTop 5 rows (Spark parquet):")
            df.show(5, truncate=False)

    def _build_zip_deliverable(self) -> str:
        """
        Input:
          - None
        Output:
          - zip_path: path to a zip file containing outputs + state
        """
        zip_path = os.path.join(self._config.base_dir, "deliverable_covid_challenge.zip")

        # Ensure we don't append to an old zip
        if os.path.exists(zip_path):
            os.remove(zip_path)

        def _iter_files(root: str) -> Iterable[Tuple[str, str]]:
            for dirpath, _, filenames in os.walk(root):
                for f in filenames:
                    abs_path = os.path.join(dirpath, f)
                    rel_path = os.path.relpath(abs_path, self._config.base_dir)
                    yield abs_path, rel_path

        with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
            for folder in [self._config.spark_parquet_dir, self._config.pandas_parquet_dir, self._config.state_dir]:
                if os.path.exists(folder):
                    for abs_path, rel_path in _iter_files(folder):
                        zf.write(abs_path, arcname=rel_path)

            # Also include a lightweight README with run instructions
            readme_text = self._render_readme_text()
            readme_path = os.path.join(self._config.base_dir, "README_DELIVERABLE.txt")
            with open(readme_path, "w", encoding="utf-8") as f:
                f.write(readme_text)
            zf.write(readme_path, arcname="README_DELIVERABLE.txt")

        return zip_path

    def _render_readme_text(self) -> str:
        """
        Input:
          - None
        Output:
          - A short README string for the zip deliverable.
        """
        return (
            "Sr Data Engineer Challenge - Covid-19 (Kaggle)\n"
            "================================================\n\n"
            "This ZIP contains:\n"
            "- parquet_spark/: Parquet outputs generated with Spark using an RDD-based CSV read.\n"
            "- parquet_pandas/: Parquet outputs generated with Pandas.\n"
            "- state/: Delta control state (seen row key hashes) so each run processes only novelties.\n\n"
            "Key design notes:\n"
            "- Type enforcement is done at the last layer before writing Parquet.\n"
            "- Delta logic uses a SHA-256 hash of a composite key per dataset.\n"
            "- Runs are append-only for delta records; state is overwritten with the merged set of keys.\n"
        )

    def _print_der_diagram(self) -> None:
        """
        Input:
          - None
        Output:
          - None. Prints a DER diagram suggestion in Mermaid format.
        Notes:
          - The Kaggle files are analytical snapshots rather than a normalized OLTP model.
          - For a clean relational model, we separate dimensions and fact tables.
        """
        print("\n============================================================")
        print("DER (Mermaid) - Suggested Analytical Model")
        print("============================================================\n")

        mermaid = r"""
derDiagram
    DIM_DATE {
      date DATE PK
    }

    DIM_REGION {
      region_id STRING PK
      country STRING
      province STRING
      who_region STRING
      continent STRING
      lat DOUBLE
      lon DOUBLE
      population BIGINT
    }

    FACT_DAILY_REGION_METRICS {
      date DATE FK
      region_id STRING FK
      confirmed BIGINT
      deaths BIGINT
      recovered BIGINT
      active BIGINT
      new_cases BIGINT
      new_deaths BIGINT
      new_recovered BIGINT
    }

    FACT_COUNTRY_LATEST {
      region_id STRING FK
      confirmed BIGINT
      deaths BIGINT
      recovered BIGINT
      active BIGINT
      new_cases BIGINT
      new_deaths BIGINT
      new_recovered BIGINT
      confirmed_last_week BIGINT
      week_change BIGINT
      week_pct_increase DOUBLE
    }

    DIM_DATE ||--o{ FACT_DAILY_REGION_METRICS : "has"
    DIM_REGION ||--o{ FACT_DAILY_REGION_METRICS : "tracks"
    DIM_REGION ||--o{ FACT_COUNTRY_LATEST : "summarizes"
"""
        print(mermaid)
        print(
            "How to use it:\n"
            "- Paste the Mermaid block into a Mermaid editor (or Markdown that supports Mermaid)\n"
            "- This is a recommended normalized analytical model based on the dataset semantics.\n"
        )

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Kaggle credentials already present.


In [11]:
# ============================================================
# Execute the pipeline
# ============================================================

# IMPORTANT: Keep the dataset id stable (matches the challenge requirement)
DATASET_ID = "imdevskp/corona-virus-report"

config = AppConfig(base_dir="/content/covid_challenge")
downloader = KaggleDownloader(dataset=DATASET_ID)
catalog = DatasetCatalog()
spark_factory = SparkSessionFactory()

runner = PipelineRunner(
    config=config,
    downloader=downloader,
    catalog=catalog,
    spark_factory=spark_factory,
)

runner.run()

Dataset URL: https://www.kaggle.com/datasets/imdevskp/corona-virus-report
License(s): other
Extracted dataset to: /content/covid_challenge/raw/extracted
Running ingestion (Spark + RDD) and ingestion (Pandas)
Delta logic: only novelties are appended each run

--- Spark report (RDD-based CSV read) ---
[Spark] covid_19_clean_complete: read=49068 | delta_written=49068
[Spark] country_wise_latest: read=187 | delta_written=187
[Spark] day_wise: read=188 | delta_written=188
[Spark] full_grouped: read=35156 | delta_written=35156
[Spark] usa_county_wise: read=627920 | delta_written=627920
[Spark] worldometer_data: read=209 | delta_written=209

--- Pandas report ---
[Pandas] covid_19_clean_complete: read=49068 | delta_written=49068
[Pandas] country_wise_latest: read=187 | delta_written=187
[Pandas] day_wise: read=188 | delta_written=188
[Pandas] full_grouped: read=35156 | delta_written=35156
[Pandas] usa_county_wise: read=627920 | delta_written=627920
[Pandas] worldometer_data: read=209 | delta_